# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# Импорт необходимых библиотек
import pandas as pd # Данные

from pymystem3 import Mystem # Лемматизация
m = Mystem()

from collections import Counter # Счетчик

In [2]:
# Считываю датасет в переменную df
df = pd.read_csv('/datasets/data.csv')

# Получаю информацию о датафрейме
df.info()

# Вывожу первые десять строк датафрейма
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Общая информация о таблице**

Итак, в таблице 12 столбцов.

Согласно документации к данным:

<table border: 1px solid grey>
  <tr>
    <th scope="col"></th>
    <th scope="col">Название столбца </th>
    <th scope="col">Расшифровка столбца </th>
    <th scope="col">Тип данных </th>
  </tr>
  <tr>
    <td>1</td>
    <td><div align="left">children</div></td>
    <td><div align="left">количество детей в семье</div></td>
    <td><div align="left">Целочисленный (int64)</div></td>
  </tr>
  <tr>
    <td>2</td>
    <td><div align="left">days_employed</div></td>
    <td><div align="left">общий трудовой стаж в днях</div></td>
    <td><div align="left">Вещественный (float64) </div></td>
  </tr>
    <tr>
      <td>3</td>
    <td><div align="left">dob_years</div></td>
    <td><div align="left">возраст клиента в годах</div></td>
    <td><div align="left">Целочисленный (int64)</div></td>
  </tr>
    <tr>
      <td>4</td>
    <td><div align="left">education</div></td>
    <td><div align="left">уровень образования клиента</div></td>
    <td><div align="left">object</div></td>
  </tr>
  <tr>
    <td>5</td>
    <td><div align="left">education_id</div></td>
    <td><div align="left">идентификатор уровня образования</div></td>
    <td><div align="left">Целочисленный (int64)</div></td>
  </tr>
  <tr>
    <td>6</td>
    <td><div align="left">family_status</div></td>
    <td><div align="left">семейное положение</div></td>
    <td><div align="left">object</div></td>
  </tr>
  <tr>
    <td>7</td>
    <td><div align="left">family_status_id</div></td>
    <td><div align="left">идентификатор семейного положения</div></td>
    <td><div align="left">Целочисленный (int64)</div></td>
  </tr>
  <tr>
    <td>8</td>
    <td><div align="left">gender</div></td>
    <td><div align="left">пол клиента</div></td>
    <td><div align="left">object</div></td>
  </tr>
  <tr>
    <td>9</td>
    <td><div align="left">income_type</div></td>
    <td><div align="left">тип занятости</div></td>
    <td><div align="left">object</div></td>
  </tr>
  <tr>
    <td>10</td>
    <td><div align="left">debt</div></td>
    <td><div align="left">имел ли задолженность по возврату кредитов</div></td>
    <td><div align="left">Целочисленный (int64)</div></td>
  </tr>
  <tr>
    <td>11</td>
    <td><div align="left">total_income</div></td>
    <td><div align="left">ежемесячный доход</div></td>
    <td><div align="left">Вещественный (float64)</div></td>
  </tr>
  <tr>
    <td>12</td>
    <td><div align="left">purpose</div></td>
    <td><div align="left">цель получения кредита</div></td>
    <td><div align="left">object</div></td>
  </tr>
</table>
</n>

В названиях колонок отсутсвуют нарушения стиля.

**Вывод**

В каждой строке таблицы — данные о клиентах банка. Часть колонок описывают социальный статус клиента: семейный статус, количество детей, образование. Также описан их доход, прошлые задолжности и цель кредита.

Предварительно можно утверждать, что данных достаточно для проверки гипотез.

Чтобы двигаться дальше, нужно произвести предобработку данных.

## Шаг 2. Предобработка данных

Для дальнейшей работы займемся предобработкой данных. Она включает в себя:  
1. Анализ данных на наличие пропусков. Действия по их устранению,
2. Замены типов данных для более комфортной работы с данными,
3. Удалению дубликатов
4. Категоризацию данных, в том числе, с использование лемм.

### Обработка пропусков

Узнаем, сколько пропусков имеет наш датафрейм

In [3]:
# Вывожу общее кол-во пропусков в каждом столбце
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Как мы видим, в столбцах `'days_employed'`(общий трудовой стаж в днях) и `'total_income'`(ежемесячный доход) имеются пропуски, равные по количеству.
На этот счет у нас теории:
1. Значения в этих столбцах случайно не внесены
2. Клиенты банка не работали и не имели доходов

Создадим таблицу, со столбцами `'days_employed'`(общий трудовой стаж в днях), `'dob_years`'(возраст клиента в годах), `'total_income`'(ежемесячный доход) и `'debt'`(имел ли задолженность по возврату кредитов)

In [4]:
# Фильтрую датафрейм по пропускам. Вывожу дополнительные столбцы
df_filter=df[df['days_employed'].isna() == True][['dob_years', 'days_employed', 'total_income', 'income_type']].sort_values(by=['dob_years'], ascending=False)
display(df_filter)

,dob_years,days_employed,total_income,income_type
18664,73,NaN,NaN,пенсионер
10563,72,NaN,NaN,пенсионер
13864,72,NaN,NaN,компаньон
6537,71,NaN,NaN,пенсионер
8456,71,NaN,NaN,пенсионер
...,...,...,...,...
12403,0,NaN,NaN,сотрудник
19829,0,NaN,NaN,сотрудник
6670,0,NaN,NaN,пенсионер
4064,0,NaN,NaN,компаньон


Проверим, есть ли в столбце `'days_employed'` датафрейма df, значения, равные 0. 

In [5]:
# Фильтрую датафрейм df по отсутствию рабочего стажа == 0
df_filter = df[df['days_employed']==0]
display(df_filter.shape)

(0, 12)

Выведим уникальные значения в столбце `'income_type'`

In [6]:
print(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


Среди уникальных значений типа занятости всего 2 значения 'безработный'. Выведим информацию по зарплате этой категории занятости

In [7]:
print(df[df['income_type'] == 'безработный'][['income_type','total_income', 'days_employed']])

       income_type   total_income  days_employed
3133   безработный   59956.991984  337524.466835
14798  безработный  202722.511368  395302.838654


Исходя из того, что столбец занятости заполнен как для людей с доходом, так и для людей без дохода, делаем выводы:  
- пропуски в столбцах о трудовом стаже и доходе случайны,
- доходы в категории 'безработные' случайны.  

На основании данных выводов:
- пропуски по доходу заменить на медианный по категориям,
- дохдодам безработных присвоить 0.  

Значения в столбце `'dob_years'`(возраст клиента в годах) равные 0 не важны, т.к. возраст не входит в исследования. Но, на всякий случай, заменим эти значения медианным возрастом всех клиентов.

Создадим словарь с уникальными значениями типа занятости и их медианным доходом

In [8]:
# Словарь, в к-ром мы будем хранить значения 'тип занятости':'медианный доход'
income_type_median = dict()

# перебираем все значения столбцов датафрейма
for i in df['income_type'].value_counts().index:
    income_type_median[i] = (df[df['income_type'] == i]
                             ['total_income'].median())

# проверяем его заполненность
for i in income_type_median.keys():
    print(i,' -', income_type_median[i])

сотрудник  - 142594.39684740017
компаньон  - 172357.95096577113
пенсионер  - 118514.48641164352
госслужащий  - 150447.9352830068
безработный  - 131339.7516762103
предприниматель  - 499163.1449470857
в декрете  - 53829.13072905995
студент  - 98201.62531401133


Заменим все пропуски столбца дохода на медианный доход по категориям

In [9]:
# перебираем все типы занятости в словаре
for i in income_type_median.keys():
    
# выделяем строки в которых фильтруются значения по пропускам в доходе и типу занятости
    df.loc[(df['total_income'].isna() == True) & (
        df['income_type'] == i), 'total_income'] = income_type_median[i]

# Проверка работы

# Проверка на отсутствие пропусков
print(df.isna().sum())

# Проверка на замену дохода на медиану по категориям
for i in income_type_median.keys():
    print(i,' -', income_type_median[i])
print('===================')


print(df.groupby('income_type')['total_income'].value_counts().head(10))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64
сотрудник  - 142594.39684740017
компаньон  - 172357.95096577113
пенсионер  - 118514.48641164352
госслужащий  - 150447.9352830068
безработный  - 131339.7516762103
предприниматель  - 499163.1449470857
в декрете  - 53829.13072905995
студент  - 98201.62531401133
income_type  total_income 
безработный  59956.991984       1
             202722.511368      1
в декрете    53829.130729       1
госслужащий  150447.935283    147
             29200.077193       1
             31483.257153       1
             34355.210785       1
             39154.156961       1
             39314.854259       1
             40264.635728       1
Name: total_income, dtype: int64


Мы убедились, что замена доходов по типу занятости прошли успешно.

Заменим пропущенные значения в столбце 'возраст' на медианные значения.  
т.к. значения трудового стажа не важны для исследования, то заменим пропуски на 0

In [10]:
# Заменяем пропущенные значения на 0. Эти значения осталист лишь в столбцах 'возраст'
# И трудовой стаж. Трудовой стаж, для нашего исследования не важен.
df = df.fillna(0)

# Перепроверяю на наличие пропусков. .sum().sum() посчитает общее количество пропусков
print('Общее количество пропусков:', df.isna().sum().sum())

# Заменяю на медианный возраст.
age_median = int(df['dob_years'].median())
print('Средний возраст:', age_median)

# Заменяю на средний возраст
df['dob_years'] = df['dob_years'].replace(0, age_median)

# Проверяю на отсутствие 0
print("Общее количество 0 в столбце 'возраст:'", df[df['dob_years'] == 0]['dob_years'].count())

Общее количество пропусков: 0
Средний возраст: 42
Общее количество 0 в столбце 'возраст:' 0


**Вывод**

Замена пропусков в датафрейме произведена.  
Анализ пропусков показал, что пропуски в датафрейме носили случайный характер. Для решения этой проблемы мы прибегли к присвоению клиентам медианного дохода по категориям.  
Данные готовы к дальнейшим действиям.

### Замена типа данных

В нашем датасете, имеются столбцы `'days_employed'`(общий трудовой стаж в днях) и `'total_income'`(ежемесячный доход ) - тип вещественный.  
т.к. количество дней не может быть дробным(может, но в нашем случае это ни к чему) и значения дробной части дохода нам тоже не важны, то для удобной работы приведем данные к целочисленному типу.

In [11]:
# Изменяю тип данных в столбцах c помощью метода astype
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')

В нашем датафрейме имеются отрицательные значения данных. Скорее всего это случайная ошибка, произошедшая при наборе данных. Заменим все отрицательные значения в датасете на положительные

In [12]:
# Перебираем названия столбцов в цикле. Заменяем отрицательные значения на положительные abs(модуль)
# Используется конструкция try-except для предотвращения возникновения ошибки в случае, если
# цикл примет значения столбца со строковым типом данных
for i in df.columns:
    try:
        df[i] = abs(df[i])
    except:
        pass

# Проверяю изменения
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

На данном этапе были изменены типы данных. Сделано это для лучшей обработки и интерпритации данных.

### Обработка дубликатов

Перед поиском дубликатов, приведем строковые данные к нижнему регистру.

In [13]:
# Перебираем названия столбцов в цикле
# Используется конструкция try-except для предотвращения возникновения ошибки в случае, если
# цикл примет значения столбца со числовым типом данных
for i in df.columns:
    try:
        df[i] = df[i].str.lower()
    except:
        pass

Необходимо проверить результат работы. Для этого выведем информацию о датафрейме и уникальные значения одного из столбцов

In [14]:
df.info()
print('===============')
print(df['education'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Проверим количество дубликатов и удалим их

In [15]:
# Промеряем количество дубликатов
print('количество дубликатов:', df.duplicated().sum())

количество дубликатов: 72


Удаляем все дубликаты

In [16]:
# Удаляем дубликаты
df = df.drop_duplicates().reset_index(drop=True)

# Проверяем
print('количество дубликатов:', df.duplicated().sum())

количество дубликатов: 0


**Вывод**

Все данные переведены к нижнему регистру, все отрицательные значения приведены к положительным.  
Дубликаты удалены. Появление дубликатов, в большей степени, вызваны пропусками в заполнении даннных о клиентах банка.

### Лемматизация

Очевидным столбцом, который необъодимо лемматизировать для дальнейшей категоризации, является столбец `'purpose'`(цель получения кредита). Для начала посмотрим уникальные значения этого столбца

In [17]:
print(df['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Проведем лемматизацию этого столбца

In [18]:
# Сохраняю значения лемм. Методу .lemmatize(<строка>) передается строка. Для корректной работы метода
# используется метод <разделитель>.join(), который склеивает слова и ставить разделитель м/у ними.
# В нашем случае пробел
lemmas = m.lemmatize(' '.join(df['purpose']))

Посчитаем колличество упоминаний всех уникальных лемм

In [19]:
# Выводим на экран леммы. метод dict.keys() выводит ключи словаря, образованного функцией Counter()
print((Counter(lemmas).keys()))

dict_keys(['покупка', ' ', 'жилье', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'на', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'со', 'заниматься', 'сделка', 'получение', 'высокий', 'подержанный', 'профильный', 'сдача', 'ремонт', '\n'])


Как мы можем убедиться, леммами, характеризующие цели кредита являются:
* операции с недвижимостью: жилье,  недвижимость, жилой
* операции с автомобилем: автомобиль
* проведение свадебных торжеств: со свадебными тожествами: свадьба
* получение образования: образование


Для простоты, объединим цели кредита в категории: недвижимость, автомобиль, свадьба, образование

In [20]:
"""def category_purpose(row):

# Возвращает названия категорий целей кредита, используя найденные леммы.

    # лемматизирую каждую строку столбца
    purpose = m.lemmatize(row['purpose'])
    
    # Проверяю вхождение слов к леммам и возвращаю категорию
    if ('жилье' in purpose or
            'недвижимость' in purpose or
            'жилой' in purpose):
        return 'недвижимость'

    elif 'автомобиль' in purpose:
        return 'автомобиль'

    elif 'свадьба' in purpose:
        return 'свадьба'

    else:
        return 'образование'

# Применяею функцию 'category_purpose' к датафрейму, чтобы получить новsй столбец с категориями целей кредита.
# axis = 1 означает что на вход в функцию отправляются строки
df['purpose_category'] = df.apply(category_purpose, axis=1)"""

def category_purpose(purpose):

# Возвращает названия категорий целей кредита, используя найденные леммы.

    # лемматизирую каждую строку столбца
    purpose = m.lemmatize(purpose)
    
    # Проверяю вхождение слов к леммам и возвращаю категорию
    if ('жилье' in purpose or
            'недвижимость' in purpose or
            'жилой' in purpose):
        return 'недвижимость'

    elif 'автомобиль' in purpose:
        return 'автомобиль'

    elif 'свадьба' in purpose:
        return 'свадьба'

    else:
        return 'образование'

# Применяею функцию 'category_purpose' к датафрейму, чтобы получить новsй столбец с категориями целей кредита.
# axis = 1 означает что на вход в функцию отправляются строки
df['purpose_category'] = df['purpose'].apply(category_purpose)

Проверим изменения в нашей таблице

In [21]:
# Подсчитываем количество пропусков
print(df.isna().sum().sum())

# Выводим уникальные значения
print(df['purpose_category'].value_counts())

display(df.head())

0
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_category, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Была произведена категоризация целей кредита с помощью лемматизации столбца `'purpose'`. Данные о категориях выведены в отдельный столбец `'purpose_category'`

### Категоризация данных

Для дальнейшей работы необходимо произвести дополнительную категоризацию данных по семейному положению, уровню дохода и наличие детей. Эти данные необходимы для ответа на вопросы данного проекта.

**Семейное положение**  
Так, данные о семейном положении представлены следующими столбцами: `'family_status'`, `'family_status_id'`.  
Выведем их уникальные значения:

In [22]:
print(df.groupby('family_status_id')['family_status'].value_counts())

family_status_id  family_status        
0                 женат / замужем          12339
1                 гражданский брак          4150
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 не женат / не замужем     2810
Name: family_status, dtype: int64


Исследуя данные значения, приходим к выводу, что проведение категоризации по семейному положению не требуется.

**Наличие детей**  
Категоризируем данные по наличию детей

In [23]:
def category_children(row):

    children = row['children']

    # Проверяю наличие детей
    if children != 0:
        return 'есть дети'

    else:
        return 'детей нет'

df['category_children'] = df.apply(category_children, axis=1)

Проверим результат работы.

In [24]:
# Подсчитываем количество пропусков
print(df.isna().sum().sum())

# Выводим уникальные значения
print(df['category_children'].value_counts())

display(df.head())

0
детей нет    14090
есть дети     7363
Name: category_children, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,category_children
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,детей нет


**Уровень дохода:**  
Для категоризации дохода обратимся к <a href="https://zavtra.ru/blogs/eto_interesno_i_polezno_znat_">статье</a> и на ее основе создадим шкалу уровня дохода.

*Уровень дохода:*  
Крайне низкий – душевой доход до 10 т.р

Очень низкий - душевой доход от 10 до 30 т.р. 

Низкий - доход от 30 до 70 т.р.

Средний – доход от 70 до 110 т.р.

Выше среднего - доход от 110 до 170 т.р.

Высокий - доход от 170 т.р.

Создадим новый столбец с категорией дохода

In [25]:
def category_total_income(row):

    income = row['total_income']

    # Проверяю уровень дохода
    if income <= 30000:
        return '1_очень низкий'

    elif income <= 70000:
        return '2_низкий'

    elif income <= 110000:
        return '3_средний'

    elif income <= 170000:
            return '4_выше среднего'

    else:
        return '5_высокий'

df['total_income_category'] = df.apply(category_total_income, axis=1)

Проверим результат работы.

In [26]:
# Подсчитываем количество пропусков
print(df.isna().sum().sum())

# Выводим уникальные значения
print(df['total_income_category'].value_counts())

display(df.head())

0
4_выше среднего    8093
5_высокий          7718
3_средний          4168
2_низкий           1452
1_очень низкий       22
Name: total_income_category, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,category_children,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,есть дети,5_высокий
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,4_выше среднего
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,детей нет,4_выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,есть дети,5_высокий
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,детей нет,4_выше среднего


**Дополнительная категоризация. Статус задолженности**  
Для построения сводных таблиц, необходим еще один столбец: категоризация задолженности.

In [27]:
def category_debt(row):

    debt = row['debt']

    # Проверяю наличие задолженности по стольцу 'debt'
    if debt == 1:
        return '0_задолженность, кол-во'

    else:
        return '1_нет задолженности, кол-во'

df['debt_status'] = df.apply(category_debt, axis=1)

**Вывод**

Данные необходимые для исследований были категоризованы и готовы к дальнейшему применению

## Шаг 3. Ответьте на вопросы

Создадим функцию, генерирующую сводные таблицы и высчитывающую:
- процент количества кридитов по ктегориям от общего количества кредитов
- процент задолжности по категориям от общего количества кредитов
- конверсию от числа задолжности. Чем выше, тем выше вероятность просрочки/задолжности по кредиту

In [28]:
def get_pivot_table(row_category):
    """ Фуркция создает сводную таблицу по категориям и
    на ее основе высчитывает процент задолжности в данной категории """

    # Создаем сводную таблицу
    some_table = df.pivot_table(
        index='debt_status',
        columns=[row_category],
        values='debt',
        aggfunc='count')

    # Создаем словарь
    # a_dict = {'категория' : ['кол-во при задолж.' , 'кол-во при отсут. задолж.']}
    lst = []
    a_dict = dict()

    for c in some_table.columns:
        lst = []
        a_dict[c] = lst

        for i in some_table.index:
            lst.append(some_table.loc[i, c])

    # Высчитываем общее количество кредитов в таблице total_credit_category
    total_credit_category = 0
    for i in a_dict.values():
        total_credit_category += sum(i)

    # формируем  словарь-строку poke_dict_category(% кредита по категориям),
    # для добавления его в конец сводной таблицы
    poke_dict_category = dict()
    for k, v in a_dict.items():
        poke_dict_category[k] = (sum(v) / total_credit_category * 100)

    # формируем  словарь-строку poke_dict_debtor(% должников),
    # для добавления его в конец сводной таблицы
    poke_dict_debtor = dict()
    for k, v in a_dict.items():
        # poke_dict_debtor[k] = ((v[0]/(v[0]+v[1])*100))
        poke_dict_debtor[k] = (v[0] / total_credit_category * 100)

    # формируем  словарь-строку poke_dict_convers(конверсия),
    #для добавления его в конец сводной таблицы
    poke_dict_convers = dict()
    for k in poke_dict_debtor.keys():
        poke_dict_convers[k] = poke_dict_debtor[k] / poke_dict_category[k] * 100
        

    # добавляем словарь-строку poke_dict_category в таблицу
    some_table.loc['2_процент категории'] = poke_dict_category

    # добавляем словарь-строку poke_dict_debtor в таблицу
    some_table.loc['3_процент задолжности по категории'] = poke_dict_debtor

    # добавляем словарь-строку poke_dict_convers в таблицу
    some_table.loc['4_конверсия'] = poke_dict_convers

    # округляем все значения до 2-го знака
    some_table = some_table.round(decimals=2)

    # выводим таблицу на экран
    display(some_table.sort_index(ascending=True))


**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [29]:
get_pivot_table('category_children')

category_children,детей нет,есть дети
debt_status,,
"0_задолженность, кол-во",1063.00,678.00
"1_нет задолженности, кол-во",13027.00,6685.00
2_процент категории,65.68,34.32
3_процент задолжности по категории,4.96,3.16
4_конверсия,7.54,9.21


**Вывод**

Наличие детей показало:
- клиенты с детьми реже берут кредиты,
- клиенты с детьми *чаще имеют задолжности* по кредиту, среди задолжников.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [30]:
get_pivot_table('family_status')

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
debt_status,,,,,
"0_задолженность, кол-во",85.00,63.00,388.00,931.00,274.00
"1_нет задолженности, кол-во",1110.00,896.00,3762.00,11408.00,2536.00
2_процент категории,5.57,4.47,19.34,57.52,13.10
3_процент задолжности по категории,0.40,0.29,1.81,4.34,1.28
4_конверсия,7.11,6.57,9.35,7.55,9.75


**Вывод**

Наличие официального брака показало:

* женатые клиенты намного чаще берут кредиты,
* женатые кленты имеют среднюю задолжность относительно живущих гражданским браком и холостых

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [31]:
get_pivot_table('total_income_category')

total_income_category,1_очень низкий,2_низкий,3_средний,4_выше среднего,5_высокий
debt_status,,,,,
"0_задолженность, кол-во",2.00,99.00,354.00,709.00,577.00
"1_нет задолженности, кол-во",20.00,1353.00,3814.00,7384.00,7141.00
2_процент категории,0.10,6.77,19.43,37.72,35.98
3_процент задолжности по категории,0.01,0.46,1.65,3.30,2.69
4_конверсия,9.09,6.82,8.49,8.76,7.48


**Вывод**

По уровню дохода:
* клиенты с доходом от выше среднего самые частые клиента банка, а так же самые частыне должники.
* говорить, о людях с низким доходом проблематично из-за маленькой выборки.

**- Как разные цели кредита влияют на его возврат в срок?**

In [32]:
get_pivot_table('purpose_category')

purpose_category,автомобиль,недвижимость,образование,свадьба
debt_status,,,,
"0_задолженность, кол-во",403.00,782.00,370.00,186.00
"1_нет задолженности, кол-во",3903.00,10029.00,3643.00,2137.00
2_процент категории,20.07,50.39,18.71,10.83
3_процент задолжности по категории,1.88,3.65,1.72,0.87
4_конверсия,9.36,7.23,9.22,8.01


**Вывод**

Ссамыми востребовательными целями, в порядке убывания являются:
1. недвижимость, с отрывом в 50%,
2. покупка/ремонт автомобиля и образование,
3. свадьба.

Анализ по целям кредитования установил их приоритет по общей задолжности:  
1. покупка/ремонт автомобиля и образование,
2. проведение свадебных торжеств,
3. седлки с недвижимостью

## Шаг 4. Общий вывод

1. Была произведена работа по анализу, предобработке и категоризации данных.
2. В процессе анализа были выявлены пропуски и ошибки в заполнении данных клиентов. Было сделано предположение о их природе и проведена работа по устранению. Без уточнения о природе данных артефактов, дальнейшие выводы могли быть не точными.
3. Данные были категоризированы, в соответствии с целями исследования.
4. Составлены таблицы, отвечающие запросам исследования. Дана краткая интерпритация полученных результатов.